<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/outputParser_240211_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set environment

In [1]:
# !pip install -q -U langchain langchain-openai

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# colab 환경설정
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [4]:
# !pip install -q icecream

In [5]:
from icecream import ic
import pprint

# Set LLMs

In [6]:
# !pip install -q -U langchain-openai

In [7]:
from langchain_openai import OpenAI
compbot = OpenAI()

In [8]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-3.5-turbo'
chatbot = ChatOpenAI(
    model_name = MODEL,
    temperature=0.7,
    max_tokens = 300
)

# Prompt

프롬프트에는 최소 2종류가 있는데,
- `PromptTemplate` : 일방적인 대답을 요구하는  
- `ChatPromptTemplate : `대화를 요구하는(그래서 나중에 히스토리로 쌓거나 할 때도 도움이 되는)

`from_template`로 틀을 잡고, `formate_prompt`로 값을 넣어주면 완성됨.
그리고 format된 결과물을 llm에 넘겨주면 대답을 해준다.

## Completion

In [9]:
# from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

# prompt_template = PromptTemplate.from_template('{subject}에 대한 재미있는 한문장짜리 농담을 해볼래?')
# prompt = prompt_template.format_prompt(subject='패션')

# res_comp = compbot.invoke(prompt)

In [10]:
# ic(prompt_template) # PromptTemplate(input_variables=['subject'], template='{subject}에 대한 재미있는 한문장짜리 농담을 해볼래?')
# ic(prompt) # StringPromptValue(text='패션에 대한 재미있는 한문장짜리 농담을 해볼래?')
# prompt.to_string() # 패션에 대한 재미있는 한문장짜리 농담을 해볼래?
# ic(res_comp)

### 아래처럼 구성할 수도 있다. 사실 이쪽을 더 많이 쓴다.
template를 구성하면서 다양한 variable을 줄 수 있고, 이 변수값들은 여러가지 단계를 거치면서 채워질 수 있기 때문이다.(이를테면 문서검색결과, 웹검색결과, 사용자입력 등... )

In [11]:
# template = '''
# 다음 주제에 대한 재미있는 한문장짜리 농담을 해볼래?

# <주제>
# {subject}
# '''

# prompt_template_02 = PromptTemplate(
#     input_variables=['subject'],
#     template = template
# )

# prompt_02 = prompt_template_02.format(subject = '패션')

# res_comp_02 = compbot.invoke(prompt_02)

In [12]:
# ic(prompt_template_02)
# ic(prompt_02) # HumanMessage가 생성되는 것에 유의.
# prompt_02
# ic(res_comp_02) # AIMessage로 돌아오는 것에 유의.

In [13]:
# chat_prompt_template = ChatPromptTemplate.from_template('{subject}에 대한 재미있는 한문장짜리 농담을 해볼래?')
# # chat_prompt = chat_prompt_template.format_messages(subject = '군고구마')
# chat_prompt = chat_prompt_template.format_prompt(subject = '군고구마')

# res_chat = chatbot.invoke(chat_prompt)

In [14]:
# ic(chat_prompt_template)
# ic(chat_prompt) # HumanMessage가 생성되는 것에 유의.
# chat_prompt.to_string
# ic(res_chat) # AIMessage로 돌아오는 것에 유의.

## Chat

사용방법  
1. 전달할 template를 만들고  
`SystemMessagePromptTemplate.from_template(system_setting)`  
2. 그걸 prompt로 만들고    
`ChatPromptTemplate.from_messages([instruction, question])`  
3. 그 prompt에 query를 넣어서  
`chat_prompt = chat_prompt_template_02.format_prompt(subject = '군고구마').to_messages()`  
4. 발사.  
`invoke()`

잘못된 예 :  
- 최종 prompt는 format_prompt로 다듬어줘야 함.  
`chat_prompt = chat_prompt_template_02.format(subject = '군고구마')`  

In [15]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

# from langchain.schema import (
#     SystemMessage,
#     HumanMessage,
#     AIMessage
# )

In [16]:
system_setting = '다음 주제에 대한 재미있는 한문장짜리 농담을 해볼래?'
instruction = SystemMessagePromptTemplate.from_template(system_setting)

# user_input = input('주제를 알려주세요.')
user_input = '{subject}'
question = HumanMessagePromptTemplate.from_template(user_input) # 사용자 입력부는, 가장 나중에 전달될 것이므로 '변수' 그 자체로 전달.

chat_prompt_template_02 = ChatPromptTemplate.from_messages([instruction, question]) # 위에서 각 만든 부품들을 조립해서 하나의 template로 만들고,..단 1개밖에 전달이 안되므로 배열로 감싸서 던진다.

chat_prompt_02 = chat_prompt_template_02.format_prompt(subject = '군고구마').to_messages() # 거기다가 마지막 사용자 질문을 입력해서 프롬프트를 완성.

# res_chat_03 = chatbot.invoke(chat_prompt_02)

In [17]:
# ic(instruction)
# ic(question)
# ic(chat_prompt_template_02)
# ic(chat_prompt_02)
# ic(res_chat_03)
# print('\n\n', res_chat_03.content )

In [18]:
import random

system_msg = SystemMessagePromptTemplate.from_template('사용자가 전달하는 숫자에 다음의 숫자를 더해줘.')
repeat = str(random.randrange(0,50))
user_input = '16'
human_msg = HumanMessagePromptTemplate.from_template(user_input)
prompt_full = ChatPromptTemplate.from_messages([system_msg, repeat, human_msg])

# res_full = chatbot.invoke(prompt_full.format_prompt())
# res_full

# Prompt Engineering

## Few-shot

백문이 불여일견이라는 말은 AI에게도 통하는 모양이다. 내가 원하는 결과물의 형태가 있는 경우, 구질구질 설명하지 말고 깔끔하게 샘플을 만들어서 보내주면 그걸 응용해서 결과물을 뽑아준다.

In [19]:
examples = [
    {
      "question": "초등학교때는 어땠어?",
      "answer": """
        초등학교때 선생님은 너무 좋았어.
        하지만 애들이 나를 싫어했었어.
        그래서 별로 재미가 없었어.
        """,
    },
    {
      "question": "중학교때는 어땠어?",
      "answer": """
        중학교때 선생님은 좀 싫었어.
        하지만 애들이 나를 많이 좋아했었어.
        그래서 연애도 많이 했지.
        """,
    },
    {
        "question": "고등학교때는 어땠어?",
        "answer": """
        고등학교때 선생님은 예뻤어.
        애들도 많이 좋아했었어.
        그래서 나도 공부를 열심히 했지.
        """,
    }
]

In [20]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

prompt_template = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\nAnswer:{answer}"
)

# print(prompt_template.format(**examples[0]))

In [21]:
few_shot_prompt = FewShotPromptTemplate( # 여기에서 비로소 FewShot~이 등장한다.
    examples=examples,
    example_prompt=prompt_template,
    suffix="Question: {input}", # suffix의 의미는... 'input값은 **이따가** 줄건데, 그건 Question 옆에다가 배치해줘...'
    input_variables=["input"],
)

print(few_shot_prompt.format(input="대학교때는 어땠어?"))

Question: 초등학교때는 어땠어?
Answer:
        초등학교때 선생님은 너무 좋았어. 
        하지만 애들이 나를 싫어했었어. 
        그래서 별로 재미가 없었어.
        

Question: 중학교때는 어땠어?
Answer:
        중학교때 선생님은 좀 싫었어.
        하지만 애들이 나를 많이 좋아했었어.
        그래서 연애도 많이 했지.
        

Question: 고등학교때는 어땠어?
Answer:
        고등학교때 선생님은 예뻤어.
        애들도 많이 좋아했었어. 
        그래서 나도 공부를 열심히 했지.
        

Question: 대학교때는 어땠어?


In [22]:
# chatbot.invoke(few_shot_prompt.format(input="대학교때는 어땠어?"))

## Example selector

세상에 질문은 많고, 인간의 감정을 '배워야만 알 수 있는' AI한계는 분명하다. 다양한 유형의 요구에, 붙어 앉아서 적절한 example set을 먹여줘야 원하는 결과를 얻을 수 있다는 건 어불성설이니, 몇가지 예시덩어리를 던져주고, 사용자의 요구에 따라 적절한 예시덩어리를 '선택'하여 활용할 수 있도록 해야 한다.  

단, 이 '선택하는 행위'를 LLM이 직접 하게되면, 그 많은 예시덩어리를 다 일단 넘겨서(자원낭비, 리스크컨트롤) 죄다 한번씩 훑어보게 해야 할 것이다. 그래서 중간관리자가 살펴보고 적절한 예시덩어리를 골라서 최종처리자 LLM에게 던져줄 필요가 발생한다.

- 이게 느낌상... router같은데...
- agent tool 이랑 langgraph 같기도 하고?

In [23]:
# !pip install -q chromadb tiktoken

### Completion

#### 1. 내가 원하는 건 이거야.

In [24]:
# 최종적으로 내가 원하는 답변 템플릿 구성.
example_prompt = PromptTemplate(
    input_variables = ['input', 'output'],
    template='input: {input}\nOutput:{output}'
)

#### 2. 네가 사용할 수 있는 자료들

In [25]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "정치인들에 대해서 어떻게 생각해?", "output": "정치이야기는 싫어"},
    {"input": "사내정치에 대해서 어떻게 생각해?", "output": "정치이야기는 싫어"},
    {"input": "종교인들에 대해서 어떻게 생각해?", "output": "종교이야기는 싫어"},
    {"input": "결혼생활에 대해서 어떻게 생각해?", "output": "결혼은 하면 미친짓, 안하면 바보짓이야"},
    {"input": "대학생활에 대해서 어떻게 생각해?", "output": "대학은 가면 자원낭비 안가면 인생낭비야"},
    {
        "input": "나비에 대한 노래를 불러줘",
        "output": "나비야 나비야 이리날아 오너라.",
    },
    {
        "input": "학교 종에 대한 노래를 불러줘",
        "output": "학교종이 떙떙땡 어서 모이자",
    },
]

#### 3. 너를 도와주실 매니저님을 소개할께.

In [26]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [27]:
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [28]:
my_manager = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
# my_manager.select_examples({"input": "병아리"})

'''
# 이렇게 통으로 해버리는 것도 가능은 하지만 좋은 예는 아닌듯?
# 계속 embedding을 반복할 테니.
 my_manager_total = SemanticSimilarityExampleSelector.from_examples(
     examples,
     OpenAIEmbeddings(),
     Chroma,
     k=1
 )
'''

'\n# 이렇게 통으로 해버리는 것도 가능은 하지만 좋은 예는 아닌듯?    \n# 계속 embedding을 반복할 테니.  \n my_manager_total = SemanticSimilarityExampleSelector.from_examples(\n     examples,\n     OpenAIEmbeddings(),\n     Chroma,\n     k=1\n )\n'

#### 4. 그러니까 말이지...

##### 매니저가 없는 경우

In [29]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="병아리"))

input: 2+2
Output:4

input: 2+3
Output:5

input: 2+4
Output:6

input: 정치인들에 대해서 어떻게 생각해?
Output:정치이야기는 싫어

input: 사내정치에 대해서 어떻게 생각해?
Output:정치이야기는 싫어

input: 종교인들에 대해서 어떻게 생각해?
Output:종교이야기는 싫어

input: 결혼생활에 대해서 어떻게 생각해?
Output:결혼은 하면 미친짓, 안하면 바보짓이야

input: 대학생활에 대해서 어떻게 생각해?
Output:대학은 가면 자원낭비 안가면 인생낭비야

input: 나비에 대한 노래를 불러줘
Output:나비야 나비야 이리날아 오너라.

input: 학교 종에 대한 노래를 불러줘
Output:학교종이 떙떙땡 어서 모이자

Question: 병아리


##### 매니저가 있는 경우

In [30]:
question = "병아리"
selected_examples = my_manager.select_examples({"question": question})
# print(f"Examples most similar to the input: {question}")
# for example in selected_examples:
#     print("\n")
#     for k, v in example.items():
#         print(f"{k}: {v}")


prompt_comp = FewShotPromptTemplate(
    example_selector = my_manager, # 매니저님이 골라주시는 샘플을 참고해서
    example_prompt = example_prompt, # 내가 처음에 물었던 대로 대답해주면 되.
    prefix='사용자의 질문유형에 맞는 대답을 해줘~', # 이건 optional.
    suffix='input:{input}', # 내 질문이 뭔지는 이따가 알려줄텐데
    input_variables =['input'] # input 이라는 자리에 넣어서 잘 생각해봐.
)

In [31]:
ic(prompt_comp.format(input='병아리'))

ic| prompt_comp.format(input='병아리'): ('사용자의 질문유형에 맞는 대답을 해줘~
                                     '
                                      '
                                     '
                                      'input: 나비에 대한 노래를 불러줘
                                     '
                                      'Output:나비야 나비야 이리날아 오너라.
                                     '
                                      '
                                     '
                                      'input: 학교 종에 대한 노래를 불러줘
                                     '
                                      'Output:학교종이 떙떙땡 어서 모이자
                                     '
                                      '
                                     '
                                      'input:병아리')


'사용자의 질문유형에 맞는 대답을 해줘~\n\ninput: 나비에 대한 노래를 불러줘\nOutput:나비야 나비야 이리날아 오너라.\n\ninput: 학교 종에 대한 노래를 불러줘\nOutput:학교종이 떙떙땡 어서 모이자\n\ninput:병아리'

In [32]:
compbot.invoke(prompt_comp.format(input='병아리'))

'에 대한 노래를 불러줘\nOutput:병아리병아리 꼬끼오 이리와 저리와 병아리야'

### Chat (아마도 여기서 RAG로 바로 이어지겠지)

In [33]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

#### completion과 동일한 부분.

In [34]:
# 내가 원하는건 이거야
example_chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# 네가 사용할 수 있는 자료는 이거야
# examples
# 너를 도와주실 매니저님을 소개할께.
# my_manager

Define the few-shot prompt.
```python
selected_example = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=my_manager,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ), # example_prompt를 따로 정의하지 않고 바로 여기서 넘기는 것도 가능.
)
```

##### 매니저가 없는 경우

In [35]:
# 그래서 결론적으로 다시 정리하자면...
few_shot_chat = FewShotChatMessagePromptTemplate(
    example_prompt = example_chat_prompt, # 내가 처음에 물었던 대로 대답해주면 되.
    examples = examples, # 자료를 참고해서
    # prefix='', # 이건 optional.
    # suffix='Input:{input}\nOutput:', # 내 질문이 뭔지는 이따가 알려줄텐데
    input_variables =['input'] # input 이라는 자리에 넣어서 잘 생각해봐.
)

In [36]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 질문유형에 맞는 대답을 해줘~"),
        few_shot_chat,
        ("human", "{input}"),
    ]
)

In [37]:
chatbot.invoke(final_prompt.format(input='병아리'))

AIMessage(content='AI: 병아리 병아리 삐약 삐약')

##### 매니저가 있는 경우

In [38]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [39]:
# 그래서 결론적으로 다시 정리하자면...
few_shot_chat_manager = FewShotChatMessagePromptTemplate(
    example_prompt = example_chat_prompt,
    example_selector=my_manager,
    # prefix='사용자의 질문유형에 맞는 대답을 해줘~', # 이건 optional. > 사용불가.
    # suffix='Input:{input}\nOutput:', # 내 질문이 뭔지는 이따가 알려줄텐데 > 사용불가
    input_variables =['input'] # input 이라는 자리에 넣어서 잘 생각해봐.
    # example_prompt=ChatPromptTemplate.from_messages(
    #     [("human", "{input}"), ("ai", "{output}")]
    # ),
)

In [40]:
my_manager.select_examples({"input": "병아리"})

[{'input': '나비에 대한 노래를 불러줘', 'output': '나비야 나비야 이리날아 오너라.'},
 {'input': '학교 종에 대한 노래를 불러줘', 'output': '학교종이 떙떙땡 어서 모이자'}]

In [41]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 질문유형에 맞는 대답을 해줘~"),
        few_shot_chat_manager,
        ("human", "{input}"),
    ]
)

In [42]:
chatbot.invoke(final_prompt.format(input='병아리'))

AIMessage(content='AI: 병아리 병아리 꼬끼오, 작은 날개 펄럭이며 행복하게 살아요~')

# Output Parser

특정하게 원하는 출력형태가 있는 경우, 파서를 사용해서 고정해줄 수 있다. 만약에... API등을 구성한다면, 고정된 형태로 반환값이 와야 하기때문에... 또는 다음 과정에서 보다 편안하게 받아쓸 수 있도록 하기 위한 용도로 사용된다.   
무엇보다... 파서를 쓰는게 베스트 프랙티스라고 하니 꼭 쓰자.

In [53]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import OpenAI, ChatOpenAI

In [54]:
output_parser = CommaSeparatedListOutputParser()

In [74]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [75]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

In [76]:
res = compbot.invoke('패션영화 5개를 추천해줘')

만약에 프롬프트에 변수를 심어주고 싶다면... 아마도 `prompt.format({variable})`으로 다듬어 줘야겠징.

In [77]:
res

'\n\n1. "Devil Wears Prada" (2006): 뉴욕 패션 매거진 편집장의 조연으로 일하게 된 젊은 여성의 성장과 패션 산업의 내부를 다루는 작품.\n\n2. "Coco Before Chanel" (2009): 유명한 디자이너 코코 샤넬의 전성기 이전의 삶과 패션에 대한 열정적인 여성의 이야기.\n\n3. "The September Issue" (2009): 패션 매거진 Vogue의 2007년 9월호를 준비하는 과정과 편집장 애나 윈투어의 열정과 집착을 담은 다큐멘터리.\n\n4. "Yves Saint Laurent" (2014): 가장 영향력 있는 디자이너 중 하나인 이브 생 로랑의 성장과 업적'

In [78]:
output_parser.parse(res)

['1. "Devil Wears Prada" (2006): 뉴욕 패션 매거진 편집장의 조연으로 일하게 된 젊은 여성의 성장과 패션 산업의 내부를 다루는 작품.\n\n2. "Coco Before Chanel" (2009): 유명한 디자이너 코코 샤넬의 전성기 이전의 삶과 패션에 대한 열정적인 여성의 이야기.\n\n3. "The September Issue" (2009): 패션 매거진 Vogue의 2007년 9월호를 준비하는 과정과 편집장 애나 윈투어의 열정과 집착을 담은 다큐멘터리.\n\n4. "Yves Saint Laurent" (2014): 가장 영향력 있는 디자이너 중 하나인 이브 생 로랑의 성장과 업적']

In [79]:
output_parser.invoke(res)

['1. "Devil Wears Prada" (2006): 뉴욕 패션 매거진 편집장의 조연으로 일하게 된 젊은 여성의 성장과 패션 산업의 내부를 다루는 작품.\n\n2. "Coco Before Chanel" (2009): 유명한 디자이너 코코 샤넬의 전성기 이전의 삶과 패션에 대한 열정적인 여성의 이야기.\n\n3. "The September Issue" (2009): 패션 매거진 Vogue의 2007년 9월호를 준비하는 과정과 편집장 애나 윈투어의 열정과 집착을 담은 다큐멘터리.\n\n4. "Yves Saint Laurent" (2014): 가장 영향력 있는 디자이너 중 하나인 이브 생 로랑의 성장과 업적']

## with LCEL

왜 LCEL을 사용하면, 아웃풋이 아래쪽으로 떨어지는 예쁜 포멧으로 나오고, 그냥 parser를 부르면 위 처럼 한줄로 가는지 모르겠다.

일단, 리스트로는 바뀌지만, 아마도 콤마가 안나오는 것 같다...왜????

In [80]:
# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | compbot
output = prompt_and_model.invoke({"query": "패션영화 5개를 추천해줘"})


In [81]:
output_parser.invoke(output)

['1. Devil Wears Prada',
 '2. Crazy Rich Asians',
 '3. Zoolander',
 '4. Coco Before Chanel',
 '5. Sex and the City']

In [82]:
output_parser.parse(output)

['1. Devil Wears Prada',
 '2. Crazy Rich Asians',
 '3. Zoolander',
 '4. Coco Before Chanel',
 '5. Sex and the City']

# Todo

- [ ] 다음의 차이점은?  
`chat_prompt = chat_prompt_template.format_messages(subject = '군고구마')`  
`chat_prompt = chat_prompt_template.format_prompt(subject = '군고구마')`

- [ ] 프롬프트 템플릿만으로도 구조화가 되는데, 굳이 스키마를 같이 쓰는 이유는 무엇인가. 정말 모르겠다.

In [ ]:
이걸 LCEL 로 변경하고 싶음...

# chain = few_shot_prompt | chatbot()
# input = '대학교때는 어땠어?'
# # chain.invoke({"input": "대학교때는 어땠어?"})
# chain.invoke(input.to_messages())

In [67]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

chatbot.invoke(final_prompt.format(input='7+7'))

Human: 2+2
AI: 4
Human: 2+3
AI: 5


AIMessage(content='AI: Ah, a delightful challenge! The sum of 7 and 7 is 14.')

여기서 final_prompt를 뜯어보면 말이지.. 이런 복잡한 모양이래.
```python
ChatPromptTemplate(
    input_variables=['input'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                template='사용자의 질문유형에 맞는 대답을 해줘~'
            )
        ),
        FewShotChatMessagePromptTemplate(
            examples=[
                {'input': '2+2', 'output': '4'},
                {'input': '2+3', 'output': '5'},
                {'input': '2+4', 'output': '6'},
                {'input': '정치인들에 대해서 어떻게 생각해?', 'output': '정치이야기는 싫어'},
                {'input': '사내정치에 대해서 어떻게 생각해?', 'output': '정치이야기는 싫어'},
                {'input': '종교인들에 대해서 어떻게 생각해?', 'output': '종교이야기는 싫어'},
                {'input': '결혼생활에 대해서 어떻게 생각해?', 'output': '결혼은 하면 미친짓, 안하면 바보짓이야'},
                {'input': '대학생활에 대해서 어떻게 생각해?', 'output': '대학은 가면 자원낭비 안가면 인생낭비야'},
                {'input': '나비에 대한 노래를 불러줘', 'output': '나비야 나비야 이리날아 오너라.'},
                {'input': '학교 종에 대한 노래를 불러줘', 'output': '학교종이 떙떙땡 어서 모이자'}
            ],
            input_variables=['input'],
            example_prompt=ChatPromptTemplate(
                input_variables=['input', 'output'],
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['input'],
                            template='{input}'
                        )
                    ),
                    AIMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['output'],
                            template='{output}'
                        )
                    )
                ]
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['input'],
                template='{input}'
            )
        )
    ]
)
```